In [ ]:
import dataset

from dataset import FusedSentenceMichigan



DATASET =FusedSentenceMichigan(feature_options={},dataset_size=500)



In [ ]:
from dataset import run_segmentation

SEGMENTATION_RESULTS, DATASET_PROPERTIES = run_segmentation(dataset=DATASET)

In [ ]:
DATASET_PROPERTIES

In [ ]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import processes_segementation_results_global
PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET = processes_segementation_results_global(SEGMENTATION_RESULTS)

In [ ]:
PROCESSED_SEGEMENTATION_DATASET

In [ ]:
import pandas as pd
def convert_segmentations_to_index(segmentations, convert_fn = lambda x: x.replace("5", "4")):
    finalResults = []
    for i in range(len(segmentations)):
        results_entry = segmentations.iloc[i]
        word_count = results_entry['word_count']
        sentence = results_entry['sentence_results']
        mappings = results_entry['mappings']
        split_sentence = sentence.split("_")
        split_sentence = [convert_fn(x) for x in split_sentence]
        for j in range(len(split_sentence)):
            word_id = split_sentence[j] 
            toneclass = int(word_id[-1])
            main_Idx = i
            word_Idx = j
            mapped_onset = mappings[j]
            finalResults.append((word_id, toneclass, main_Idx, word_Idx,mapped_onset))

        
    return pd.DataFrame(finalResults, columns=['word_id', 'toneclass', 'main_Idx', 'word_Idx', 'mapped_onset'])




PSEG_INDEX = convert_segmentations_to_index(PROCESSED_SEGEMENTATION_DATASET)
PSEG_INDEX

In [ ]:
import librosa
def get_audio_sample_at_idx(idx,pSEG_index, pSEG, pSEGAUDIO, sr= 16000):
    sample_info = pSEG_index.iloc[idx]
    pSEG_idx = sample_info['main_Idx']

    sentence_info = pSEG.iloc[pSEG_idx]
    pSEG_audio = pSEGAUDIO.iloc[pSEG_idx]["audio"]


    segementation_start_time_gt = sentence_info['delimiter_time_results']
    segementation_start_time = sorted(sentence_info['segementation_times_results'] + [segementation_start_time_gt[sentence_info['word_count']]])
    segmentation_onset_mapping = sentence_info['mappings']
    mapped_onset = sample_info['mapped_onset']

    current_mapping = segmentation_onset_mapping[sample_info['word_Idx']]

    if current_mapping == -1:
        # unmapped, use previous result
        for i in range(sample_info['word_Idx'], -1, -1):
            if segmentation_onset_mapping[i] != -1:
                prev_mapping = segmentation_onset_mapping[i]
                break
        # prev_mapping = segmentation_onset_mapping[sample_info['word_Idx'] - 1]
        start_time = segementation_start_time[prev_mapping]
        end_time = segementation_start_time[prev_mapping+1] 
        print(prev_mapping,prev_mapping+1)
    else:
        start_time = segementation_start_time[current_mapping]
        end_time = segementation_start_time[current_mapping+1]

    start_samples = librosa.time_to_samples(start_time, sr=sr)
    end_samples = librosa.time_to_samples(end_time, sr=sr)

    return sample_info,pSEG_audio[start_samples:end_samples]
print(get_audio_sample_at_idx(1, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET))
# print(get_audio_sample_at_idx(720, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET))

In [ ]:
importlib.reload(dataset)
from dataset import DatasetMichigan
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

INFO,AUDIO_SAMPLE = get_audio_sample_at_idx(8, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET)
MEL = DatasetMichigan.preproccessData(AUDIO_SAMPLE, features={"mel_spectrogram"},pad_audio=True, pad_samples = librosa.time_to_samples(1, sr=16000), sampling_rate = 16000)[1]

# Plot the mel spectrogram
print(INFO)
plt.figure(figsize=(10, 4))
librosa.display.specshow(MEL, sr=16000, hop_length=321, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.show()

Audio(AUDIO_SAMPLE, rate=16000)